## 코로나 :  집단감염 현황 네트워크 분석

- $활용 데이터:$ Case, PatientInfo <br>
- $활용 라이브러리:$ pyvis <br>

### 1. 준비.

In [ ]:
# 필요한 라이브러리를 설치한다.
!pip install pyvis

In [ ]:
# 필요한 라이브러리를 불러온다.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyvis.network import Network
from IPython.core.display import display, HTML

In [ ]:
from google.colab import drive
# 구글 드라이브 마운트.
drive.mount('/content/drive')  # 절차를 따라서 한다

In [ ]:
%cd /content/drive/MyDrive/GwangjuAI/

In [ ]:
# 데이터를 가져온다.
case = pd.read_csv(r'./data/Case.csv')
patinfo = pd.read_csv(r'./data/PatientInfo.csv')

In [ ]:
case.head(3)

In [ ]:
patinfo.head(3)

In [ ]:
# 결측치 확인.
print(case.isnull().sum().sum())
print(patinfo.isnull().sum().sum())

### 2. 데이터 전처리.

In [ ]:
# 집단감연 유형 정리.
# 집단감염 유형이 아닌것들은 삭제한다.
infect_case = list(set(case['infection_case']) - set(['etc','contact with patient','overseas inflow']))

In [ ]:
# patinfo의 결측치를 문자열 'nan'으로 채워 넣어 둔다.
patinfo.fillna('nan', inplace=True)

In [ ]:
# 확진자 리스트와 감염경로 리스트.
patient_id=list(patinfo['patient_id'].apply(lambda x: str(x)))   # 문자열로 변환.
patient_case=list(patinfo['infection_case'])

In [ ]:
# 누구로부터 감염됐는가?
whom=list(patinfo['infected_by'])       # 문자열 리스트.

In [ ]:
# 전체 확진자들 중 whom이 결측치가 아니며 patient_id에 속하는 경우의 인덱스 리스트.
whom_index=[i for i in range(len(patient_id)) if whom[i] !='nan' and whom[i] in patient_id ]

In [ ]:
# 집단감염장소 사이에는 edge로 연결되지 않는다.
# 집단감염장소 -> 확진자 edge 생성을 위한 list of tuples.
center_patient=[(patient_case[i],int(patient_id[i])) for i in range(len(patient_id)) if patient_case[i] in infect_case]
# 집단감염의 경우 감염장소와 확진자 리스트를 별도로 구별해서 저장해 둔다.
center_list=list(set([x[0] for x in center_patient]))    # 고유한 집단감염장소 리스트. 짧음!
patient_list=[x[1] for x in center_patient]              # 집단감염을 통해서 감염된 환자 리스트.

In [ ]:
# 확진자 -> 확진자 edge 생성을 위한 list of tuples.
patient_list_str = [str(x) for x in patient_list]
patient_patient=[(int(whom[i]),int(patient_id[i]))for i in whom_index if (whom[i] in patient_list_str) and (patient_id[i] in patient_list_str)]

### 3. Network 시각화 생성.

In [ ]:
# Network 객체 생성.
g = Network(height=600,width=900,directed=True,notebook=True,heading='Network Graph')
g.set_options("""
var options = {
  "nodes": {
    "font": {
      "size": 100,
      "strokeColor": "rgba(165,215,255,1)"}}}
""")

In [ ]:
# 집단감염장소 별표로 추가.
for x in center_list:
    g.add_node(n_id=x,color='grey',label=x, shape='star')   

In [ ]:
# 개별환자 점으로 추가.
for x in patient_list:
    g.add_node(n_id=x,label=[''],shape='dot' ) 

In [ ]:
# 집단감염장소와 개별환자 사이를 edge로 이음.
for center,patient in center_patient:
    g.add_edge(source=center,to=patient, color='green', width=1)    # width는 edge의 강도를 나타냄.

In [ ]:
# 개별환자와 개별환자 사이를 edge로 이음.
for patient1, patient2 in patient_patient:
    g.add_edge(source=patient1,to=patient2, color='red', width=1)   # width는 edge의 강도를 나타냄.

In [ ]:
# 출력.
g.show('my_network.html')
display(HTML('my_network.html'))